In [1]:
# !pip uninstall sklearn -y
# !pip install --upgrade sklearn
# !pip install scikit-learn==0.23.1 --user
# !pip install pytorch_tabnet
# !pip install optuna

     ------------------------------------- 365.3/365.3 kB 23.7 MB/s eta 0:00:00
     ------------------------------------- 210.5/210.5 kB 13.4 MB/s eta 0:00:00
     ---------------------------------------- 78.7/78.7 kB ? eta 0:00:00


In [1]:
import pandas as pd
import random
import os
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold, StratifiedKFold

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from pytorch_tabnet.tab_model import TabNetClassifier
from pytorch_tabnet.metrics import Metric

import optuna
from optuna import Trial, visualization

In [27]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
class F1_Macro(Metric):
    def __init__(self):
        self._name = "f1"
        self._maximize = True
    def __call__(self, y_true, y_score):
        y_pred = np.argmax(y_score, axis = 1)
        score = f1_score(y_true, y_pred, average = "macro")
        return score

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

In [4]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [5]:
train_x = train_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP', 'Y_Quality'])

test_x = test_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])

In [6]:
# qualitative to quantitative

qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_x[i])
    train_x[i] = le.transform(train_x[i])
    
    for label in np.unique(test_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_x[i] = le.transform(test_x[i]) 
print('Done.')

Done.


In [7]:
# na 80%이상 열 삭제

col = train_x.columns
col = col[3:]
for i in col:
    if train_x.loc[:, i].isna().sum() > len(train_x) * 0.8:
        train_x.drop(i, axis = 1, inplace = True)

In [8]:
# test에서도 동일하게 삭제

test_x = test_x.loc[:, train_x.columns[1:]]

In [11]:
# # 결측치 0으로 채움
# train_x = train_x.fillna(0)
# test_x = test_x.fillna(0)

밑의 코드는 이산형 변수와 연속형 변수를 나름 임의대로 분리한 과정이다.
feature의 unique 값이 12개 이하이면 이산형 변수로 그 외에는 연속형 변수로 판단하였다.
unique값의 기준을 나름 다시 잘 재정의할 필요가 있다.

In [9]:
idx = train_x.columns[1:]
feat_discrete = []
feat_numeric = []
for i in idx:
    check = train_x.loc[:, i]
    if len(np.unique(check)) + 1 <= 13:
        feat_discrete.append(i)
    else:
        feat_numeric.append(i)

simpleimputer를 이용하였다. 더 좋은 방법이 있으면 그것을 사용할 예정이다.
simpleimpute의 default로는 내가 알기로는 mean값이 default로 채워진다.
이산형 변수에 대해서는 strategy로 최빈값을 연속형 변수에는 default인 mean으로 채워 넣었다.
impute를 우선 trainset으로 학습을 시키고 동일하게 testset에도 적용시켜 Data Leakage 문제에 저촉되지 않도록 하였다

In [10]:
from sklearn.impute import SimpleImputer
imp_continuous = SimpleImputer()
imp_discrete = SimpleImputer(strategy = "median")
target = train_x["Y_Class"]
train_x.drop("Y_Class", axis = 1, inplace = True)
imp_continuous.fit(train_x.loc[:, feat_numeric])
imp_discrete.fit(train_x.loc[:, feat_discrete])
imputed_df_train_num = pd.DataFrame(imp_continuous.transform(train_x.loc[:, feat_numeric]), columns = feat_numeric)
imputed_df_train_dis = pd.DataFrame(imp_discrete.transform(train_x.loc[:, feat_discrete]), columns = feat_discrete)
imputed_df_train = pd.concat([imputed_df_train_num, imputed_df_train_dis], axis = 1)
imputed_df_test_num = pd.DataFrame(imp_continuous.transform(test_x.loc[:, feat_numeric]), columns = feat_numeric)
imputed_df_test_dis = pd.DataFrame(imp_discrete.transform(test_x.loc[:, feat_discrete]), columns = feat_discrete)
imputed_df_test = pd.concat([imputed_df_test_num, imputed_df_test_dis], axis = 1)

In [11]:
add_cols = train_x.iloc[:, 0:2]
add_col = test_x.iloc[:, 0:2]
imputed_df_train = pd.concat([add_cols, imputed_df_train], axis = 1)
imputed_df_test = pd.concat([add_col, imputed_df_test], axis = 1)

In [12]:
train = imputed_df_train
train["Y_Class"] = target
test = imputed_df_test

In [13]:
idx = train.columns
col = []
# 하나의 값만 들어있는 column 들은 나가리 시킨다.
for i in idx:
    check = train.loc[:, i]
    if len(np.unique(check)) == 1:
        col.append(i)

In [14]:
train.drop(col, axis = 1, inplace = True)
test.drop(col, axis = 1, inplace = True)

In [15]:
# train set의 label encoding을 통해 알 수 있는 점: 이 2개의 column은 범주형 자료다.
num_unique = np.array(train.nunique())
categorical_columns = ["LINE", "PRODUCT_CODE"]
categorical_dims = {"LINE": num_unique[0], "PRODUCT_CODE": num_unique[1]}
cat_idxs = [i for i, f in enumerate(train.columns) if f in categorical_columns]
cat_dims = [categorical_dims[f] for i, f in enumerate(train.columns) if f in categorical_columns]

In [22]:
train_y = train['Y_Class']
train_x = train.drop(columns=['Y_Class'])

In [23]:
# split dataset
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.2, stratify=train_y, random_state=42)

In [24]:
train_x = train_x.to_numpy()
train_y = train_y.to_numpy()
valid_x = valid_x.to_numpy()
valid_y = valid_y.to_numpy()

In [25]:
cat_idxs = [0, 1]
cat_dims = [6, 3]

In [28]:
clf = TabNetClassifier(cat_idxs=cat_idxs,
                       cat_dims=cat_dims,
                       cat_emb_dim=3,
                       optimizer_fn=torch.optim.Adam,
                       optimizer_params=dict(lr=1e-2),
                       scheduler_params={"step_size":50,
                                         "gamma":0.9},
                       scheduler_fn=torch.optim.lr_scheduler.StepLR,
                       mask_type='sparsemax')

In [ ]:
# max_epochs = 2000

# clf.fit(X_train=train_x, y_train=train_y,
#         eval_set=[(train_x, train_y), (valid_x, valid_y)],
#         eval_name=['train', 'valid'],
#         eval_metric=['f1'],
#         max_epochs=max_epochs, patience=10,
#         batch_size=1024, virtual_batch_size=128,
#         num_workers=0,
#         weights=1,
#         drop_last=False)

In [38]:
def Objective(trial):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    mask_type = trial.suggest_categorical("mask_type", ["entmax", "sparsemax"])
    n_da = trial.suggest_int("n_da", 16, 64, step=4)
    n_steps = trial.suggest_int("n_steps", 3, 10, step=2)
    gamma = trial.suggest_float("gamma", 1., 1.5, step=0.1)
    cat_emb_dim = trial.suggest_int("cat_emb_dim", 3, 10, step=1)
    n_shared = trial.suggest_int("n_shared", 3, 5, step=1)
    n_independent = trial.suggest_int("n_independent", 3, 5, step=1)
    lambda_sparse = trial.suggest_float("lambda_sparse", 1e-6, 1e-3, log=True)
    tabnet_params = dict(n_d=n_da,
                         n_a=n_da,
                         n_steps=n_steps,
                         gamma=gamma,
                         cat_idxs=cat_idxs,
                         cat_dims=cat_dims,
                         cat_emb_dim=cat_emb_dim,
                         lambda_sparse=lambda_sparse,
                         optimizer_fn=torch.optim.Adam,
                         optimizer_params=dict(lr=1e-2, weight_decay=1e-5),
                         mask_type=mask_type,
                         n_shared=n_shared,
                         n_independent=n_independent,
                         scheduler_params=dict(mode="min",
                                               patience=trial.suggest_int("patienceScheduler",9, 10, step=1), # changing sheduler patience to be lower than early stopping patience 
                                               min_lr=1e-5,
                                               factor=0.5,),
                        scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
                        device_name = device,
                        verbose=0
                        ) #early stopping
    kf = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
    CV_score_array = []
    for train_index, test_index in kf.split(train_x, train_y):
        X_train, X_valid = train_x[train_index], train_x[test_index]
        y_train, y_valid = train_y[train_index], train_y[test_index]
        classifier = TabNetClassifier(**tabnet_params)
        classifier.fit(X_train=X_train, y_train=y_train,
                       eval_set=[(X_train, y_train), (X_valid, y_valid)],
                       eval_name = ["train", "valid"],
                       patience=trial.suggest_int("patience", 25, 30, step=5),
                       max_epochs=trial.suggest_int('epochs', 500, 1000, step=100),
                       batch_size=24, virtual_batch_size=8,
                       eval_metric=["logloss", 'f1'])
        CV_score_array.append(classifier.best_cost)
    avg = np.mean(CV_score_array)
    return avg

In [39]:
study = optuna.create_study(direction='minimize', study_name='TabNet optimization')
study.optimize(Objective)

[I 2023-02-25 12:46:15,714] A new study created in memory with name: TabNet optimization
C:\Users\user\anaconda3\lib\site-packages\optuna\distributions.py:716: UserWarning: The distribution is specified by [3, 10] and step=2, but the range is not divisible by `step`. It will be replaced by [3, 9].
  warnings.warn(



Early stopping occurred at epoch 67 with best_epoch = 37 and best_valid_f1 = 0.57619


C:\Users\user\anaconda3\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Early stopping occurred at epoch 67 with best_epoch = 37 and best_valid_f1 = 0.46297


C:\Users\user\anaconda3\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


...


Early stopping occurred at epoch 48 with best_epoch = 23 and best_valid_f1 = 0.48823


C:\Users\user\anaconda3\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2023-02-25 21:36:53,181] Trial 11 finished with value: 0.4969029742280534 and parameters: {'mask_type': 'entmax', 'n_da': 24, 'n_steps': 5, 'gamma': 1.5, 'cat_emb_dim': 10, 'n_shared': 4, 'n_independent': 4, 'lambda_sparse': 0.0002748698113937471, 'patienceScheduler': 10, 'patience': 25, 'epochs': 800}. Best is trial 4 with value: 0.4699891534198941.
C:\Users\user\anaconda3\lib\site-packages\optuna\distributions.py:716: UserWarning: The distribution is specified by [3, 10] and step=2, but the range is not divisible by `step`. It will be replaced by [3, 9].
  warnings.warn(



Early stopping occurred at epoch 78 with best_epoch = 48 and best_valid_f1 = 0.46252


C:\Users\user\anaconda3\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[W 2023-02-25 21:42:26,536] Trial 12 failed with parameters: {'mask_type': 'entmax', 'n_da': 24, 'n_steps': 7, 'gamma': 1.4, 'cat_emb_dim': 9, 'n_shared': 5, 'n_independent': 5, 'lambda_sparse': 0.0008766879211993953, 'patienceScheduler': 9, 'patience': 30, 'epochs': 700} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\user\AppData\Local\Temp\ipykernel_5516\332815910.py", line 38, in Objective
    classifier.fit(X_train=X_train, y_train=y_train,
  File "C:\Users\user\anaconda3\lib\site-packages\pytorch_tabnet\abstract_model.py", line 241, in fit
    self._train_epoch(train_dataloader)
  File "C:\Users\user\anaconda3\lib\site-packages

KeyboardInterrupt: 

In [ ]:
TabNet_params = study.best_params

In [22]:
TabNet_params = {'mask_type': 'entmax',
                 'n_da': 64,
                 'n_steps': 5,
                 'gamma': 1.1,
                 'cat_emb_dim': 10,
                 'n_shared': 4,
                 'n_independent': 3,
                 'lambda_sparse': 0.0009570756759623438,
                 'patienceScheduler': 9,
                 'patience': 25,
                 'epochs': 300}

In [23]:
TabNet_params

{'mask_type': 'entmax',
 'n_da': 64,
 'n_steps': 5,
 'gamma': 1.1,
 'cat_emb_dim': 10,
 'n_shared': 4,
 'n_independent': 3,
 'lambda_sparse': 0.0009570756759623438,
 'patienceScheduler': 9,
 'patience': 25,
 'epochs': 300}

In [ ]:
final_params = dict(n_d=TabNet_params['n_da'], 
                    n_a=TabNet_params['n_da'],
                    n_steps=TabNet_params['n_steps'],
                    gamma=TabNet_params['gamma'],
                    cat_emb_dim=TabNet_params['cat_emb_dim'],
                    lambda_sparse=TabNet_params['lambda_sparse'],
                    optimizer_fn=torch.optim.Adam,
                    optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
                    mask_type=TabNet_params['mask_type'],
                    n_shared=TabNet_params['n_shared'],
                    scheduler_params=dict(mode="min",
                                          patience=TabNet_params['patienceScheduler'],
                                          min_lr=1e-5,
                                          factor=0.5,),
                     scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
                     verbose=1,
                     )
epochs = TabNet_params['epochs']

In [ ]:
clf = TabNetClassifier(**final_params)

kf = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)

for train_index, test_index in kf.split(train_x, train_y):
    X_train, X_valid = train_x[train_index], train_x[test_index]
    y_train, y_valid = train_y[train_index], train_y[test_index]
    clf.fit(X_train=train_x, y_train=train_y,
            eval_set=[(X_valid, y_valid)],
            patience=TabNet_params['patience'],
            max_epochs=epochs,
            batch_size=24, virtual_batch_size=8,
            eval_metric=['f1'])

In [ ]:
preds = clf.predict(test.values)

In [ ]:
np.unique(preds, return_counts=True)

In [28]:
submit = pd.read_csv('/content/drive/MyDrive/Data/sample_submission.csv')

In [29]:
submit['Y_Class'] = preds

In [30]:
submit.to_csv('/content/drive/MyDrive/Data/final_submission.csv', index=False)

In [ ]:
clf.save_model('/content/drive/MyDrive/Data/best_tabne')